# 5.1 LU Decomposition

This tutorial will not directly help you get the Class 05 in-class exercise done.  It may help you think more clearly and more efficiently about programming the LU decomposition method, and that may help you with the exercise.  But if it somehow reveals the solution to the exercise in an "Aha!" moment, then it has failed.  You must learn how to "think in loop space", with all of its indexing tricks and traps, and you will only learn that by struggling with problems again and again.  Not by seeing any answers.

We start by importing numpy.  Note that all cells are active in  jupyter notebook.  Thus, we only need to import it once and we can use it from then on throughout this notebook.  If something goes wrong and you need to restart the Kernel, be sure to rerun this cell before proceeding.

In [ ]:
import numpy as np

Since we're working with matrices, let's make a reference matrix and calculate it's inverse by hand.  For simplicity, let's make it a 2x2.  Recall that a 2x2 matrix,
$$
A
\quad = \quad
 \begin{bmatrix}
  a & b \\
  c & d
 \end{bmatrix}
$$

has an inverse

$$
A^{-1}
\quad = \quad
 \begin{bmatrix}
  a & b \\
  c & d
 \end{bmatrix}^{-1}
\quad = \quad
 \frac{1}{ad-bc}
 \begin{bmatrix}
  d & -b \\
  -c & a
 \end{bmatrix}
$$

Thus,

$$
\begin{bmatrix}
  1 & 2 \\
  3 & 4 
 \end{bmatrix}^{-1}
\quad = \quad
 \begin{bmatrix}
  -1 & \frac{3}{2} \\
   2 & -\frac{1}{2} 
 \end{bmatrix}
$$

Let's check our hand calculation.

In [ ]:
A = np.array([[1.0, 2.0], [3.0, 4.0]])
Ainv = np.array([[-2.0,1.0], [3.0/2.0, -1.0/2.0]])
print ('The A matrix is')
print (A)
print()
print ('The Ainv matrix is')
print (Ainv)
print()
print('Is A Ainv = I?')
print(A @ Ainv)

So far so good:  we have a reference matrix, and we correctly have its inverse to check our results with.

And now we stop programming!  It's pointless to throw code on the page if we don't quite know what we're doing, so let's repeat the derivation of LU factorization for a 2x2 matrix, using our own notation.

Seriously!

We know you can repeat what was done in the notes, and we know you can repeat what is done below.  But if we gave you a closed-notes exam and asked you to derive the LU factorization for even a simple 2x2, could you do it reasonably quickly?  You won't know until you try, so please try.

Then go on and compare what you wrote with what's written below, and realign your thinking as needed.

.<br />
.<br />
.<br />


Our objective is to construct a lower triangular $L$ matrix and an upper triangular $U$ matrix such that

$$
A \ = \ L\ U \quad \Rightarrow \quad L^{-1}\ A \ = \ U
$$
or
$$
 \begin{bmatrix}
  1 & 0 \\
  c_{Li} & 1 
 \end{bmatrix}
 \begin{bmatrix}
  a & b \\
  c & d 
 \end{bmatrix} \ = \ 
 \begin{bmatrix}
  a_U & b_U \\
  0   & d_U 
 \end{bmatrix}
$$

where the only thing we know is that we want 1's on the main diagonal (by design) and a 0 in the upper right corner of $L^{-1}$ (to make it lower triangular, because the inverses of all lower triangular matrices are lower triangular), and we want a zero in the lower left corner of $U$ (to make it upper triangular).

Note that we do not use $a_{11}, a_{12}, \dots$ notation.  We decided to not add the complexity of keeping track of indices in trying this out.  Later we might look back on this as a mistake, but that would be unfair.  Right now we're trying to keep things simple so we can understand the challenge in front of us; we're not trying to plan for the challenges ahead.  If we try to do too much in one step, we become paralyzed and get nothing done.  Don't let that happen to you!

We can do the matrix multiplication by hand to see that

$$
(1)(a) + (0)(c) = a_U \qquad \quad (1)(b) + (0)(d) = b_U \qquad \quad (c_{Li})(a) + (1)(c) = 0 \qquad \quad (c_{Li})(b) + (1)(d) = d_U
$$

Given $a$, $b$, $c$, and $d$, the third equation solves for $c_{Li}$, and then the fourth for $d_U$.  The first two equations show that the first row of $A$ is the same as the first row of $U$.  Substituting these solutions into the matrices above gives

$$
 \begin{bmatrix}
  1 & 0 \\
  -\frac{c}{a} & 1 
 \end{bmatrix}
 \begin{bmatrix}
  a & b \\
  c & d 
 \end{bmatrix} \ = \ 
 \begin{bmatrix}
  a & b \\
  0   & d - \frac{bc}{a} 
 \end{bmatrix}
$$

So now, given the elements of a 2x2 matrix $A$, we can compute $L^{-1}$ and $U$.  But what about $L$?  In the notes it was implied that

$$
L^{-1} \ = \ 
 \begin{bmatrix}
  1 & 0 \\
  -x & 1 
 \end{bmatrix}
\quad \Rightarrow \quad 
L \ = \ 
 \begin{bmatrix}
  1 & 0 \\
  x & 1 
 \end{bmatrix}
$$

Really?  Really?!  You won't know until you try it, so put pen to paper and try multiplying them together and see what you get.

.<br />
.<br />
.<br />


$$
 \begin{bmatrix}
  1 & 0 \\
  x & 1 
 \end{bmatrix}
 \begin{bmatrix}
  1 & 0 \\
  -x & 1 
 \end{bmatrix} \ = \ 
 \begin{bmatrix}
  1 & 0 \\
  0 & 1 
 \end{bmatrix}
$$

So it is true... at least for 2x2 matrices.  Is it true for all lower trianglular matrices with one's on the main diagonal? NO!  Try it for a 3x3 -- you'll see that if the row=3/col=2 element is not zero, you do not get the identity matrix.  But we're getting ahead of ourselves.

Now lets write some code to get the LU factorization of a 2x2 matrix.  No need to be fancy.  Let's just make sure we know what we're doing.

In [ ]:
import numpy as np    # just as a reminder

# input data
a = 1.0
b = 2.0
c = 3.0
d = 4.0

# construct the A matrix
A = np.array( [[a, b], [c, d]] )
print('The matrix A is')
print(A)
print()

# construct Linv, L, and U from the elements of A
Linv = np.array( [[1, 0], [-c/a,         1]] )  ; print('Linv is ') ; print(Linv) ; print()
L    = np.array( [[1, 0], [ c/a,         1]] )  ; print('L is '   ) ; print(L   ) ; print()
U    = np.array( [[a, b], [   0, d - b*c/a]] )  ; print('U is '   ) ; print(U   ) ; print()

# check that these are what we claim
print ('Does Linv @ L = I?')
print (Linv @ L)
print()
print ('Does L @ U = A?')
print (L @ U)
print()



If you were to write the above code, how many mistakes would you make?  None, of course, because it's so obvious and so simple.  But your narrator, an experienced programmer, left out some brackets (twice) and typed a + where a - was needed.  Even with the comments and the nice spacing and not a single index to be seen.

There are those who can write perfect code every time.  And they are very rare.  We mere mortals should use print statements to display every little thing, and with strings to tell us what we're looking at.  Yes, it takes time, but it generally saves even more time.

So we now have a code that computes the LU decomposition for a 2x2 matrix.  What about a 3x3?  A 4x4?  An n x n?

Our code is fine, and it works.  But it does not scale.

Our challenge is to perform an LU decomposition on an arbitrary square matrix (that is guaranteed to be invertible &mdash; we're not trying to trick anyone).  The good news is that we are starting from something that works.  Let's take it to the next level.  Let's rewrite everything using indices:  $a = a_{11}, b = a_{12}, c = a_{21},$ etc.  That way we can take advantage of patterns when we scale up.

We'll start by constructing the Linv matrix.  

In [ ]:
import numpy as np

# input data
a = 1.0   # A[0,0]
b = 2.0   # A[0,1]
c = 3.0   # A[1,0]
d = 4.0   # A[1,1]

# construct A matrix
A = np.array( [[a, b], [c, d]] )
print('The matrix A is')
print(A)
print()

# Construct Linv by creating a 2x2 identity matrix and then changing the 2,1 element.
# Element refers to the mathematical indices; item will refer to the Python indices.
# So the 2,1 element is the 1,0 item in Python
Linv = np.eye(2)
print('Is this really an identity matrix?')
print(Linv)
print()
Linv[1,0] = -A[1,0]/A[0,0]
print('For A with elements 1,2,3,4, the matrix Linv should be [[ 1.  0.], [-3.  1.]]')
print(Linv)
print()

Success!  Let's keep going.  (Recall that Jupyter notebooks are continuous, so as long as you ran the cell above, we can start in the middle here.)

In [ ]:
# construct L by negating the 2,1 element in Linv
L = np.eye(2)
L[1,0] = -Linv[1,0]
print('For A with elements 1,2,3,4, the matrix L should be [[ 1.  0.], [3.  1.]]')
print(L)
print()

# construct U by creating a 2x2 identity matrix, writing the first row of A into it,
# and then changing the 2,2 element to d-bc/a
U = np.eye(2)
U[0,0:2] = A[0,0:2]     # note range notation:  0:2 will give items 0 and 1 (elements 1 and 2)
U[1,  1] = A[1,1] - A[0,1] * A[1,0] / A[0,0]
print('For A with elements 1,2,3,4, the matrix U should be [[ 1.  2.], [0.  -2.]]')
print(U)
print()

# check that these are what we claim
print ('Does Linv @ L = I?')
print (Linv @ L)
print()
print ('Does L @ U = A?')
print (L @ U)
print()

So now we have a fully indexed version.  But let's notice a simplification we can make.  Note that $A[1,0] / A[0,0]$ was already computed in $Linv$.  So we can simplify the computation of $U$ a little.

In [ ]:
# construct U by creating a 2x2 identity matrix, writing the first row of A into it,
# and then changing the 2,2 element to d-bc/a
U = np.eye(2)
U[0,0:2] = A[0,0:2]     # note range notation:  0:2 will give items 0 and 1 (elements 1 and 2)
U[1,  1] = A[1,1] + A[0,1] * Linv[1,0]       # now using -c/a from Linv, being very careful about the signs
print('For A with elements 1,2,3,4, the matrix U should be [[ 1.  2.], [0.  -2.]]')
print(U)
print()

Again success.  Now let's get at a sub-challenge in our LU factorization:  not creating separate Linv, L, and U matrices.  What if we store everything in A as we make it?  Effectively, we want to transform $A$ as

$$
 \begin{bmatrix}
  a & b \\
  c & d 
 \end{bmatrix} \ \rightarrow \ 
 \begin{bmatrix}
  a & b \\
  c_L   & d_U 
 \end{bmatrix}
$$

Recall that L has 1's along its main diagonal, so the right side matrix has all of the new information we need to recreate $L$ and $U$.

In [ ]:
import numpy as np

# input data
a = 1.0   # A[0,0]
b = 2.0   # A[0,1]
c = 3.0   # A[1,0]
d = 4.0   # A[1,1]

# construct A matrix
A = np.array( [[a, b], [c, d]] )
print('The matrix A is')
print(A)
print()

# construct the revised A matrix with L and U in it, without using the Linv[1,0] shortcut
A[1,0] = A[1,0]/A[0,0]                       # L[1,0]
A[1,1] = A[1,1] - A[0,1] * A[1,0] / A[0,0]   # U[1,1]
print('For A with elements 1,2,3,4, the revised matrix A should be [[ 1.  2.], [3.  -2.]]')
print(A)
print()

Well, it looks right.  <b><u>But it's wrong!</u></b>  Ok, the answer is correct, but that's only because we got lucky.  Or, rather, unlucky.

Rerun the above with a = 2.0 instead of 1.0, and see if you get the revised $A = [[2. \ \ 2.], [1.5 \ \ 1.]]$ 

Uh oh!  What happened?  Can you spot the problem?

In creating the revised $A$ matrix, first we updated $A[1,0]$.  But in the second step we tried to use the old $A[1,0]$, even though that number has changed.  Not good.

The only reason it looked correct is because we chose a test matrix with $A[0,0] = 1$:  a bad choice in hindsight, since it clearly does not challenge our programming very well.  Keep that in mind as you test your codes:  how confident are you that you are right, and not just unlucky?

But now notice that the new $A[1,0]$ is equal to $A[1,0]/A[0,0]$, which is exactly the $Linv[1,0]$ shortcut we tried above.  We implement this fix in the next cell, again being very careful to get the signs right.

Since we overwrote A above and messed up the input data, we have to recreate the original A in the next cell before we can work on it.

In [ ]:
import numpy as np

# input data
a = 1.0   # A[0,0]
b = 2.0   # A[0,1]
c = 3.0   # A[1,0]
d = 4.0   # A[1,1]

print()# construct A matrix
A = np.array( [[a, b], [c, d]] )
print('The matrix A is')
print(A)
print()

# construct the revised A matrix with L and U in it, with the Linv[1,0] shortcut
A[1,0] = A[1,0]/A[0,0]              # L[1,0]
A[1,1] = A[1,1] - A[0,1] * A[1,0]   # U[1,1]
print('For A with elements 1,2,3,4, the revised matrix A should be [[ 1.  2.], [3.  -2.]]')
print(A)
print()

Horray!  All is well.  But one more step.  Notice the computation
$$A[1,1] \ \ = \ \ A[1,1] \ - \ A[0,1] * A[1,0]$$
This can be rewritten using a compound operator as
$$A[1,1] \ \ \ -= \ \ A[0,1] * A[1,0]$$

Some may argue that this makes the code less readable.  Others may disagree.  Regardless, you need to be able to read other people's codes no matter what their programming philosophy, and this is a fine time to practice.  So, our final 2x2 version is as follows.

In [ ]:
import numpy as np

# input data
a = 1.0   # A[0,0]
b = 2.0   # A[0,1]
c = 3.0   # A[1,0]
d = 4.0   # A[1,1]

print()# construct A matrix
A = np.array( [[a, b], [c, d]] )
print('The matrix A is')
print(A)
print()

# construct the revised A matrix with L and U in it, with the Linv[1,0] shortcut
A[1,0]  =  A[1,0] / A[0,0]   # L[1,0]
A[1,1] -=  A[0,1] * A[1,0]   # U[1,1]
print('The revised matrix A should be [[ 1.  2.], [3.  -2.]]')
print(A)
print()

But now what about 3x3 matrices?  This is more complicated because we can't get from A to the revised A in one step.  We need to formulate the intermediate M matrices first.  Working directly from the "preliminaries" notes, we have the following.

In [ ]:
# construct an A1 matrix (aka A)
A1 = np.array( [[ 1.0,  2.0,  3.0],   \
                [ 4.0, 13.0, 18.0],   \
                [ 7.0, 54.0, 78.0]] )
print('The matrix A1 is')
print(A1)
print()

M1 = np.eye(3)                # this makes a 3x3 identity
M1[1:,0] -= A1[1:,0]/A1[0,0]  # using a slice to compute items 1,0 and 2,0 (last two elements of first column)
print('The matrix M1 is')
print(M1)
print()
A2 = M1 @ A1
print('The matrix A2 is')
print(A2)
print()

M2 = np.eye(3)
M2[2:,1] -= A2[2:,1]/A2[1,1]  # using a slice to compute item 2,1 - the last element of second column of 3x3...
print('The matrix M2 is')     #      ...which is also the last element of the first column of the 2x2 submatrix
print(M2)
print()
A3 = M2 @ A2
print('The matrix A3 (aka U) is')
print(A3)
print()

# Lets do the usual checks that these are what we claim
M1inv = M1.copy() ; M1inv[1:,0] = -M1[1:,0]
M2inv = M2.copy() ; M2inv[2:,1] = -M2[2:,1]

Linv = M2    @ M1
print('The matrix Linv is')
print(Linv)
print()
L    = M1inv @ M2inv
print('The matrix L is')
print(L)
print()

U = A3.copy()
print('The matrix U is')
print(U)
print()

print ('Does Linv @ L = I?')
print (Linv @ L)
print()
print ('Does L @ U = A?')
print (L @ U)
print()

Ok, that works fine.  But again we have these intermediate $M$ matrices, and the lecture notes show us that we can store these elements directy in $A$.  So, similar to the 2x2 case, we want to transform $A$ as

$$
 \begin{bmatrix}
  a & b & c \\
  d & e & f \\
  g & h & i
 \end{bmatrix} \ \rightarrow \ 
 \begin{bmatrix}
  A1_{1,1} & A1_{1,2} & A1_{1,3} \\
  M1_{2,1} & A2_{2,2} & A2_{2,3} \\
  M1_{3,1} & M2_{3,2} & A3_{3,3}
 \end{bmatrix} \ = \
 \begin{bmatrix}
   U_{1,1} &  U_{1,2} &  U_{1,3} \\
   L_{2,1} &  U_{2,2} &  U_{2,3} \\
   L_{3,1} &  L_{3,2} &  U_{3,3}
 \end{bmatrix}
$$

We will construct each of the elements in the revised $A$ in the order we computed them above.  First the three $A1$ elements across the top, and then the two $M1$ elements down the first column -- these steps reduce the problem to decomposing the 2x2 in the lower right corner of $A$.  Next the two $A2$ elements across the top of the 2x2, and the one $M2$ down the first column of the 2x2 -- these steps effectively reduce the problem to decomposing the 1x1 in the lower right corner of $A$, which is the element from $A3$.

In [ ]:
# construct an A matrix
A = np.array( [[ 1.0,  2.0,  3.0],   \
               [ 4.0, 13.0, 18.0],   \
               [ 7.0, 54.0, 78.0]] )
print('The matrix A is')
print(A)
print()

# This was how it was done for the 2x2 case; extrapolate to the 3x3 case.
# Construct the revised A matrix with L and U in it, with the Linv[1,0] shortcut.
# A[1,0] = A[1,0]/A[0,0]              # L[1,0]
# A[1,1] = A[1,1] - A[0,1] * A[1,0]   # U[1,1]

# construct the revised A matrix
# The first row stays intact        # A[0,0:] stays the same
# first compute the M1 elements in the full 3x3 matrix
A[1,0] = A[1,0]/A[0,0]              # L[1,0] = M1[1,0]
A[2,0] = A[2,0]/A[0,0]              # L[2,0] = M1[2,0]
# now update the reduced 2x2 entries based on M1
A[1,1] = A[1,1] - A[0,1] * A[1,0]   # U[1,1] = A2[1,1]
A[1,2] = A[1,2] - A[0,2] * A[1,0]   # U[1,2] = A2[1,2]
A[2,1] = A[2,1] - A[0,1] * A[2,0]   # U[2,1] = A2[2,1]
A[2,2] = A[2,2] - A[0,2] * A[2,0]   # U[2,2] = A2[2,2]
# next compute the M2 element in the reduced 2x2 matrix
A[2,1] = A[2,1]/A[1,1]              # L[2,1] = M2[2,1]
# now update the reduced 1x1 entry based on M2
A[2,2] = A[2,2] - A[1,2] * A[2,1]   # U[2,2] = A3[2,2]

print('The new matrix "A" which contains L and U is')
print(A)
print()

This checks with what we got above.  But we can still do better.  First, we can use compound operators to simplify the x = x-y statements to just x -= y.

Second, there is a lot of element by element coding in constructing the revised $A$ matrix that could instead be handled by loops.  There will be an outer loop that starts with the 3x3 to operate on its first column, and then increments to operate on the second column &mdash; which is the first column of the reduced 2x2 matrix.  The index of this loop is referred to as k in the notes.

This k loop will, in its first loop in the 3x3, compute<br />
&emsp; $A[1,0] \ = \ A[1,0]/A[0,0]$<br />
&emsp; $A[2,0] \ = \ A[2,0]/A[0,0]$<br />
and in its second loop in the reduced 2x2 it will compute<br />
&emsp; $A[2,1] \ = \ A[2,1]/A[1,1]$<br />

<b>This is the $A[k+1:,k] \ = \ A[k+1:,k]/A[k,k]$ step in the notes.</b>

Within each k loop the sub-matrix needs to be updated.  For this 3x3, the first submatrix is the 2x2, updated as<br />
&emsp; $A[1,1] \ = \ A[1,1] - A[0,1] * A[1,0]$<br />
&emsp; $A[1,2] \ = \ A[1,2] - A[0,2] * A[1,0]$<br />
&emsp; $A[2,1] \ = \ A[2,1] - A[0,1] * A[2,0]$<br />
&emsp; $A[2,2] \ = \ A[2,2] - A[0,2] * A[2,0]$<br />
When working on the 2x2, the submatrix is a 1x1, updated as<br />
&emsp; $A[2,2] \ = \ A[2,2] - A[1,2] * A[2,1]$<br />

<b>These are the $A[i,j] \ \ -= \ A[i,k]*A[k,j]$ steps in the notes, except that in code above the factors in the multiplcation are reversed.</b>  That is, the code above reads as<br />
&emsp; $A[i,j] \ -= \ A[k,j]*A[i,k]$<br />

So your in-class assignment boils down to this.<br />
&ensp; &bull; Implement the k outer loop to compute <b>$A[k+1:,k] \ = \ A[k+1:,k]/A[k,k]$.</b><br />
&ensp; &bull; Implement the i,j inner loops to compute <b>$A[i,j] \ -= \ A[i,k]*A[k,j]$.</b><br />
Verify that you get the correct answers for the 3x3 matrix.

Then generalize your code to handle any n x n matrix.
Verify that you get the correct answers for the 3x3 and 2x2 matrices.

Finally, remove all of the print statements or set your "Submitty" flag to True to suppress any output.

QED